In [38]:
using GLMakie, LinearAlgebra

# TODO: Try CUDA computation
# TODO: Add sliders for max force, max speed and perception radious
# TODO: Maybeeeeee try make .exe 

# Include file with movement algorithm
include("boids.jl")

# Make boids
axis_limit = 700;
generation_area = 200
numBoids = 500
flock = [Boid(i, rand(-generation_area:generation_area, 3), randn(3).*3, randn(3).*2) for i=1:numBoids]
points = Observable([Point3f(boid.position) for boid in flock])

# Create figure
fig = Figure()

# Create 3d axis
ax = Axis3(fig[1, 1], aspect = (1, 1, 1), title = "Flocking simulation")
ax.perspectiveness = 0.3

# Set axis limits
limits!(ax, -axis_limit, axis_limit, -axis_limit, axis_limit, -axis_limit, axis_limit)

# Create sliders layout
lsgrid = labelslidergrid!(fig,
    ["align", "separation", "cohesion"],
    [Ref(LinRange(0.1:0.01:2)), Ref(LinRange(0.1:0.01:3)), Ref(LinRange(0.1:0.01:4))];
    formats = [x -> "$(round(x, digits = 2))"],
    labelkw = Dict([(:textsize, 20)]),
    sliderkw = Dict([(:linewidth, 10)]),
    valuekw = Dict([(:textsize, 20)])
)

# Create sliders grid
sl_sublayout = GridLayout(height=50)
fig[2, 1] = sl_sublayout
fig[2, 1] = lsgrid.layout

# Set initial parameters
set_close_to!(lsgrid.sliders[1], 0.5)
set_close_to!(lsgrid.sliders[2], 1.0)
set_close_to!(lsgrid.sliders[3], 0.6)

# Update parameters values
alignValue = lsgrid.sliders[1].value
separationValue = lsgrid.sliders[2].value
cohesionValue = lsgrid.sliders[3].value

# Create buttons grid
fig[3, 1] = buttongrid = GridLayout(height=100)

labels = ["Run / Pause", "Step", "Restart"]
button_count = 3

# Create buttons
buttons = buttongrid[3, 1:button_count] = [
    Button(fig, label=l, height=35, width=150, textsize=20)
    for l in labels
]

# Run button
isrunning = Observable(false)
on(buttons[1].clicks) do clicks; isrunning[] = !isrunning[]; end

on(buttons[1].clicks) do clicks
    @async while isrunning[]
        for boid in flock
            fly!(boid, flock, alignValue[], separationValue[], cohesionValue[], axis_limit[])
        end
        
        points[] = [Point3f(boid.position) for boid in flock]
        sleep(0.001)
    end
end

# Step button
on(buttons[2].clicks) do click
    for boid in flock
        fly!(boid, flock, alignValue[], separationValue[], cohesionValue[], axis_limit[])
    end
    
    points[] = [Point3f(boid.position) for boid in flock]
    sleep(0.001)
end

# Reset button
on(buttons[3].clicks) do click
    for boid in flock
        boid.position = rand(-generation_area:generation_area, 3)
    end
    points[] = [Point3f(boid.position) for boid in flock]
    sleep(0.001)
end

scatter!(ax, points, markersize=20, color=:black)
display(fig)

LoadError: MethodError: no method matching -(::LinRange{Float64, Int64}, ::Int64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar
[0mClosest candidates are:
[0m  -([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at C:\Users\zakamycze\AppData\Local\Programs\Julia-1.7.2\share\julia\base\int.jl:86
[0m  -([91m::CEnum.Cenum{T}[39m, ::S) where {T<:Integer, S<:Integer} at C:\Users\zakamycze\.julia\packages\CEnum\Bqafi\src\operators.jl:9
[0m  -([91m::GeometryBasics.HyperRectangle{N, T}[39m, ::Number) where {N, T} at C:\Users\zakamycze\.julia\packages\GeometryBasics\3PqdK\src\primitives\rectangles.jl:266
[0m  ...